In [155]:
from eth_utils import address
from web3 import Web3
import os
from solcx import compile_standard, install_solc
import json

version = '0.8.0'
install_solc(version)

smart_contract = "EnergyTrading.sol"

with open(smart_contract, "r") as file:
    contract_file = file.read()

compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"EnergyTrading.sol": {"content": contract_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version=version,  # type: ignore
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

# get bytecode
bytecode = compiled_sol["contracts"][smart_contract]["EnergyTrading"]["evm"]["bytecode"]["object"]

# get abi
abi = json.loads(compiled_sol["contracts"][smart_contract]["EnergyTrading"]["metadata"])["output"]["abi"]

# set up connection
w3 = Web3(Web3.HTTPProvider("HTTP://172.21.128.1:7545"))
chain_id = 1337
owner_address = "0x952b1253dF851D217501Fa66fa5f4a8cC9025Eac"
private_key = "0c1aa4f476b3a2768675e3d1190ba26df71fd2e3044e01712db59538cdb82760"

# initialize contract
EnergyTrading = w3.eth.contract(abi=abi, bytecode=bytecode)
nonce = w3.eth.get_transaction_count(owner_address)

# set up transaction from constructor which executes when firstly
transaction = EnergyTrading.constructor().build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": owner_address, 
        "nonce": nonce
    }
)

signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [156]:
# REGISTER THE USER

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"
private_key = "90c2213fbee613082e2340ed07de21e526d5b4a9b04dbc236780e799ee6bfa98"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

call_fun = energy_trading_sol.functions.registerUser().build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

print(tx_call_fun_receipt)

AttributeDict({'transactionHash': HexBytes('0x50dab6512d7c72de6ad29b136eaa43647d81c293aa8902beeef37504d27814c3'), 'transactionIndex': 0, 'blockHash': HexBytes('0xf25e496efd578544d628f82a94fb076821de1ea981898631740fc5fd928ff26a'), 'blockNumber': 120, 'from': '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530', 'to': '0x96Baa2B8399374A545a09C6B5a060081F2B53d4D', 'gasUsed': 44415, 'cumulativeGasUsed': 44415, 'contractAddress': None, 'logs': [AttributeDict({'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0x50dab6512d7c72de6ad29b136eaa43647d81c293aa8902beeef37504d27814c3'), 'blockHash': HexBytes('0xf25e496efd578544d628f82a94fb076821de1ea981898631740fc5fd928ff26a'), 'blockNumber': 120, 'address': '0x96Baa2B8399374A545a09C6B5a060081F2B53d4D', 'data': HexBytes('0x000000000000000000000000b28d5f7929aa1f5f6d5811c692d59255abfdd530'), 'topics': [HexBytes('0x54db7a5cb4735e1aac1f53db512d3390390bb6637bd30ad4bf9fc98667d9b9b9')], 'type': 'mined'})], 'status': 1, 'logsBloom': HexBytes('0x0000

In [157]:
# dump contract address to file
with open("../backend/utils/contract_address.json", "w") as file:
    json.dump({"contractAddress": tx_receipt.contractAddress}, file)

# dump contract abi to file
with open("../backend/utils/contract_abi.json", "w") as file:
    json.dump(abi, file)

In [198]:
# CREATE OFFER

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"
private_key = "90c2213fbee613082e2340ed07de21e526d5b4a9b04dbc236780e799ee6bfa98"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

call_fun = energy_trading_sol.functions.offerEnergy(
    5000, 14, 500, 50
).build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

print(tx_call_fun_receipt)

AttributeDict({'transactionHash': HexBytes('0xf6795fb2bfb41738a3de84c6c3410ef0487a68f14209b024c791a5ed14c8fe79'), 'transactionIndex': 0, 'blockHash': HexBytes('0x4571089e855f8f42720b7194dd862c0be5b5f8e1d0c9052beadaef9941bed2a7'), 'blockNumber': 144, 'from': '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530', 'to': '0x96Baa2B8399374A545a09C6B5a060081F2B53d4D', 'gasUsed': 158377, 'cumulativeGasUsed': 158377, 'contractAddress': None, 'logs': [AttributeDict({'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0xf6795fb2bfb41738a3de84c6c3410ef0487a68f14209b024c791a5ed14c8fe79'), 'blockHash': HexBytes('0x4571089e855f8f42720b7194dd862c0be5b5f8e1d0c9052beadaef9941bed2a7'), 'blockNumber': 144, 'address': '0x96Baa2B8399374A545a09C6B5a060081F2B53d4D', 'data': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000006000000000000000000000000b28d5f7929aa1f5f6d5811c692d59255abfdd53000000000000000000000000000000000000000000000000000000000000013880000000000000000000000000000

In [199]:
# GET ACTIVE OFFERS

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)
payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"
private_key = "90c2213fbee613082e2340ed07de21e526d5b4a9b04dbc236780e799ee6bfa98"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

# Call the function and get the return value
active_offers = energy_trading_sol.functions.getActiveOffers().call({
    "from": payer_address
})

active_offers

[(4,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732208099,
  '0x0000000000000000000000000000000000000000',
  500,
  False),
 (5,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732208101,
  '0x0000000000000000000000000000000000000000',
  500,
  False)]

In [ ]:
# REGISTER THE USER

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa"
private_key = "f8615fbd141bf5cc9156f3b6e21f966bbd22cd2d530b069d6033cd929700afe1"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

call_fun = energy_trading_sol.functions.registerUser().build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

print(tx_call_fun_receipt)

AttributeDict({'transactionHash': HexBytes('0xe398f2b7382ee85a2b99d52463510002d04837792b49b1d251b027d87e8c43f2'), 'transactionIndex': 0, 'blockHash': HexBytes('0xafd74c828f4f5e7daae36a86eeb0c31ab5cd1a5f0e1de2f986ff919edf5cb4bd'), 'blockNumber': 145, 'from': '0xC77B3dcc92F50d28CBb17821d56f682A9E329C05', 'to': '0x96Baa2B8399374A545a09C6B5a060081F2B53d4D', 'gasUsed': 44415, 'cumulativeGasUsed': 44415, 'contractAddress': None, 'logs': [AttributeDict({'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0xe398f2b7382ee85a2b99d52463510002d04837792b49b1d251b027d87e8c43f2'), 'blockHash': HexBytes('0xafd74c828f4f5e7daae36a86eeb0c31ab5cd1a5f0e1de2f986ff919edf5cb4bd'), 'blockNumber': 145, 'address': '0x96Baa2B8399374A545a09C6B5a060081F2B53d4D', 'data': HexBytes('0x000000000000000000000000c77b3dcc92f50d28cbb17821d56f682a9e329c05'), 'topics': [HexBytes('0x54db7a5cb4735e1aac1f53db512d3390390bb6637bd30ad4bf9fc98667d9b9b9')], 'type': 'mined'})], 'status': 1, 'logsBloom': HexBytes('0x0000

In [ ]:
# PLACE BID

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa"
private_key = "f8615fbd141bf5cc9156f3b6e21f966bbd22cd2d530b069d6033cd929700afe1"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

call_fun = energy_trading_sol.functions.placeBid(
    3
).build_transaction(
    {
        "value": w3.to_wei("1", "ether"),
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

print(tx_call_fun_receipt)

An RPC error was returned by the node. Check the message provided in the error and any available logs for more information.


Web3RPCError: {'message': "sender doesn't have enough funds to send tx. The upfront cost is: 100000000000000000 and the sender's account only has: 0", 'code': -32000, 'data': {'stack': "Error: sender doesn't have enough funds to send tx. The upfront cost is: 100000000000000000 and the sender's account only has: 0\n    at VM.<anonymous> (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\ethereumjs-vm\\lib\\runTx.ts:114:11)\n    at step (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\ethereumjs-vm\\dist\\runTx.js:33:23)\n    at Object.next (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\ethereumjs-vm\\dist\\runTx.js:14:53)\n    at fulfilled (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\

In [ ]:
# END OFFER

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"
private_key = "90c2213fbee613082e2340ed07de21e526d5b4a9b04dbc236780e799ee6bfa98"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

call_fun = energy_trading_sol.functions.endAuction(
    4
).build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

print(tx_call_fun_receipt)

ContractLogicError: ('execution reverted: VM Exception while processing transaction: revert Auction has already ended.', {'stack': 'RuntimeError: VM Exception while processing transaction: revert Auction has already ended.\n    at Function.RuntimeError.fromResults (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\lib\\utils\\runtimeerror.js:94:13)\n    at module.exports (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\lib\\utils\\gas\\guestimation.js:142:32)', 'name': 'RuntimeError'})

In [169]:
# GET BIDS BY OFFER

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"

# Call the function and get the return value
active_offers = energy_trading_sol.functions.getBidsByOffer(0).call({
    # "from": payer_address
})

active_offers

[('0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa', 200000000000000000, 0),
 ('0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa', 400000000000000000, 0),
 ('0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa', 500000000000000000, 0)]

In [175]:
# GET OFFERS BY USER

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"

# Call the function and get the return value
active_offers = energy_trading_sol.functions.getOffersByUser(payer_address).call({
    "from": payer_address
})

active_offers

[(0,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732206265,
  '0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa',
  600000000000000000,
  True),
 (1,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732206271,
  '0x0000000000000000000000000000000000000000',
  500,
  False),
 (2,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732206272,
  '0x0000000000000000000000000000000000000000',
  500,
  False)]

In [205]:
# get contract balance

w3.eth.get_balance("0x952b1253dF851D217501Fa66fa5f4a8cC9025Eac")

92963918140000000000